<a href="https://www.kaggle.com/code/syedfuzail1/google-agentic-course-project?scriptVersionId=281119583" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install -q google-genai
!pip install -q google-adk --no-deps


In [ ]:
from google.adk.agents import Agent
from google import genai

print("ADK and genai imported successfully!")


In [ ]:
print("Importing required libraries...")

import os
import asyncio

from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.tools import google_search, load_memory
from google.adk.tools.agent_tool import AgentTool
from google.genai.types import Content, Part

print("✔ Libraries imported successfully!")

# Basic app constants
APP_NAME = "conceptloop_micro_tutor"
USER_ID = "demo_user"
SESSION_ID = "session_1"

# API key setup
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = "api"  # TEMP hardcode for testing
print("✔ GOOGLE_API_KEY set!")

MODEL = "gemini-2.5-flash"
print("✔ Model set to:", MODEL)


In [ ]:
print("Initializing session and memory services...")

session_service = InMemorySessionService()
memory_service = InMemoryMemoryService()

print("✔ Session & Memory Services initialized successfully!")


In [ ]:
print("Defining custom tool and sub-agents...")

def update_progress(topic: str, step_name: str, result: str) -> str:
    return f"Progress logged for topic='{topic}', step='{step_name}', result='{result}'"

planner_agent = Agent(
    name="planner_agent",
    model=MODEL,
    description="Breaks a topic into learning steps.",
    instruction=(
        "Break the given topic into 3–5 clear steps. "
        "Format: Step <number>: <short title> - <one sentence goal>"
    )
)

tutor_agent = Agent(
    name="tutor_agent",
    model=MODEL,
    description="Explains topic steps simply.",
    instruction=(
        "Explain the given step in extremely simple words using small examples."
    ),
    tools=[google_search],
)

quiz_agent = Agent(
    name="quiz_agent",
    model=MODEL,
    description="Creates mini quiz questions.",
    instruction=(
        "Ask 1-2 small questions and check the answer."
    ),
)

print("✔ Sub-agents and tools created!")


In [ ]:
print("Wrapping sub-agents as tools and defining coach agent...")

planner_tool = AgentTool(agent=planner_agent)
tutor_tool = AgentTool(agent=tutor_agent)
quiz_tool = AgentTool(agent=quiz_agent)

coach_agent = Agent(
    name="coach_agent",
    model=MODEL,
    description="Coordinates planner, tutor, and quiz flow.",
    instruction=(
        "You are a study coach. Use planner → tutor → quiz flow. "
        "If user fails quiz, explain again and retry once. "
        "Log progress after each step."
    ),
    tools=[planner_tool, tutor_tool, quiz_tool, load_memory, update_progress],
)

print("✔ Coach agent created successfully!")


In [ ]:
print("Setting up runner and execution flow...")

async def run_once(user_message: str):
    print(f"\n🚀 Starting new learning session for: '{user_message}'")

    # delete existing session first if present
    try:
        await session_service.delete_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
        print("🗑 Previous session deleted.")
    except:
        print("ℹ No previous session found, creating new session.")

    await session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID,
        state={"current_topic": user_message}
    )
    print("✨ New session created.")

    runner = Runner(
        agent=coach_agent,
        app_name=APP_NAME,
        session_service=session_service,
        memory_service=memory_service,
    )
    print("✔ Runner initialized")

    content = Content(parts=[Part(text=user_message)], role="user")

    final_text = ""
    print("🤖 Running agent interaction...")

    # ---------------------------
    # UPDATED DEBUG EVENT VIEW
    # ---------------------------
    async for event in runner.run_async(
        user_id=USER_ID,
        session_id=SESSION_ID,
        new_message=content,
    ):
        print("\n--- EVENT RECEIVED ---")

        # Print raw event object
        print("Event Object:", event)

        # Print any content parts
        if hasattr(event, "content") and event.content:
            for part in event.content.parts:
                if hasattr(part, "text") and part.text:
                    print("📝 Text:", part.text)
                elif hasattr(part, "function_call") and part.function_call:
                    print("🛠 Function Call:", part.function_call)
                else:
                    print("🔧 Other Part:", part)

        # Capture final output when done
        if event.is_final_response() and event.content and event.content.parts:
            final_text = event.content.parts[0].text

    print("\n💾 Saving session to long-term memory...")
    session = await runner.session_service.get_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID,
    )
    await memory_service.add_session_to_memory(session)
    print("📚 Session saved.")

    print("\n🎉 FINAL AGENT RESPONSE:\n")
    print(final_text)


print("✔ Runner ready. Call run_once() to begin!")


In [ ]:
await run_once("I want to understand binary search from scratch.")


In [ ]:
await run_once("teach me about addition")

